# CLIP 

download ms coco dataset from: https://gist.github.com/mkocabas/a6177fc00315403d31572e17700d7fd9

https://medium.com/howtoai/pytorch-torchvision-coco-dataset-b7f5e8cad82

using Resnet 50, MS COCO dataset, train and val set

In [1]:
from torchvision import datasets
import torch
import clip
import numpy as np
import os

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# to use a different clip model
model, preprocess = clip.load("RN50", device=device)
#model, preprocess = clip.load('ViT-L/14', device=device)

: 

In [ ]:
model

In [ ]:
preprocess

# MS COCO

In [ ]:
path2data="/home/ubuntu/mmdetection_od/mmdetection/data/coco/images/train2017"
path2json="/home/ubuntu/mmdetection_od/mmdetection/data/coco/annotations/instances_train2017.json"

In [ ]:
train_dataset = datasets.CocoDetection(root=path2data, annFile=path2json)

In [ ]:
train_dataset

# Dataset filtering

In [ ]:
# Filter the dataset to only include images with annotations
annotated_images_indices = []
for idx in range(len(train_dataset)):
    # get the annotations for the current image
    _, target = train_dataset[idx]
    
    # check if the current image has any annotations
    if len(target) > 0:
        annotated_images_indices.append(idx)

In [ ]:
len(annotated_images_indices)

In [ ]:
with open('annotated_train_images_indices_2017.txt', 'w') as f:
    for idx in annotated_images_indices:
        f.write(str(idx) + '\n')

In [ ]:
filtered_train_dataset=torch.utils.data.Subset(train_dataset, annotated_images_indices)

In [ ]:
len(filtered_train_dataset)

Features extraction using CLIP 

In [ ]:
image_file_names=[]
image_features=[]

# iterate over the dataset and extract the filename and image features
for idx in range(len(filtered_train_dataset)):
    # get the image and its annotations
    img, target = filtered_train_dataset[idx]
    
    image =  preprocess(img).unsqueeze(0).to(device)
    
    # extract the image features
    with torch.no_grad():
        image_features.append(torch.squeeze(model.encode_image(image)))
    
    # extract the filename
    image_id = target[0]['image_id']
    image_info = train_dataset.coco.loadImgs(image_id)[0]
    image_file_names.append(image_info['file_name'])


In [ ]:
len(image_features)

In [ ]:
len(image_file_names)


In [ ]:
image_features_cpu=[image_feature.cpu().numpy() for image_feature in image_features]

# Save features

In [ ]:
#Save features in the correct results folder
dataset='mscoco'
results_folder = os.path.join('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan','results')
if not os.path.exists(results_folder):
        os.makedirs(results_folder)
results_folder_dataset = os.path.join(results_folder,dataset)
if not os.path.exists(results_folder_dataset):
        os.makedirs(results_folder_dataset)
results_folder_dataset_pretext = os.path.join(results_folder_dataset,'pretext')
if not os.path.exists(results_folder_dataset_pretext):
        os.makedirs(results_folder_dataset_pretext)
np.save(os.path.join(results_folder_dataset_pretext,'features_seed1_clip'), image_features_cpu)

In [ ]:
check_emb=np.load('/home/ubuntu/master_thesis/covering_lens/TypiClust/scan/results/mscoco/pretext/features_seed1_clip.npy')

In [ ]:
check_emb.shape

In [ ]:
with open(os.path.join(results_folder_dataset_pretext, 'filenames.txt'), 'w') as f:
    for image_file_name in image_file_names:
        f.write(image_file_name + '\n')